### **I only picked mammals, birds, reptiles and amphibians because I normally pay attention to these animals**

In [6]:
import json
import numpy as np

In [7]:
# Load the training dataset annotation json file
# "C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\train_mini.json"
try:
    with open("C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\train_mini.json", 'r') as file:
        all_data_ann = json.load(file)
    # print(train_mini)
except json.JSONDecodeError as e:
    print(f"Error decoding JSON: {e}")
except FileNotFoundError:
    print("File not found.")

In [8]:
# Pring some general information
print(len(all_data_ann["images"]))
print(all_data_ann["images"][0].keys())
print(all_data_ann["images"][0]["id"])
print('over')

500000
dict_keys(['id', 'width', 'height', 'file_name', 'license', 'rights_holder', 'date', 'latitude', 'longitude', 'location_uncertainty'])
0
over


In [40]:
# Define a function that gets the level of category from the annotations
def get_image_label(cateid: int, categories, type: str):
    # categories should look like this: test_ann['categories']
    # print(categories)
    cate_list = list(categories[0].keys())[1:]
    # print(cate_list)
    if type not in cate_list:
        return "please enter correct category level"

    # Find the dictionary with 'id' equal to 0
    target_cate = next((cate for cate in categories if cate['id'] == cateid), None)

    if target_cate is None:
        #print("Found dictionary:")
        print(f"Dictionary with 'id' equal to {cateid} not found.")
        return None

    return target_cate[type]

In [41]:
# specify the classes I want to keep
mbra = ['Mammalia', 'Aves', 'Reptilia', 'Amphibia']

**Create a new annotation only for mammals/birds/reptiles/amphibians**

In [11]:
# construct a new dictionary that contains only essential parts
mbra_data_ann = {'images': [],
                 'categories': [],
                 'annotations': []}

data_length = len(all_data_ann['images'])
cate_id_recoder = []

print(data_length)

for i in range(data_length):
    # check the class of current image
    cate_id = all_data_ann['annotations'][i]['category_id']
    # get the class name
    class_name = get_image_label(cate_id, all_data_ann['categories'], 'class')
    if class_name in mbra:
        # append image
        mbra_data_ann['images'].append(all_data_ann['images'][i].copy())
        # append categories
        if cate_id not in cate_id_recoder:
            mbra_data_ann['categories'].append(all_data_ann['categories'][cate_id].copy())
            cate_id_recoder.append(cate_id)
        # append annotations
        mbra_data_ann['annotations'].append(all_data_ann['annotations'][i].copy())

# print length to check integrity
print('total number of images for m/b/r/a: ', len(mbra_data_ann['images']))
print('total number of annotations for m/b/r/a: ', len(mbra_data_ann['annotations']))
print('total number of categories for m/b/r/a: ', len(mbra_data_ann['categories']))


500000
total number of images for m/b/r/a:  110750
total number of annotations for m/b/r/a:  110750
total number of categories for m/b/r/a:  2215


In [12]:
# concat dictionary
mbra_data_ann_tolocal = json.dumps(mbra_data_ann)

# Write to files
f = open("C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\Image\\data_annotations\\mbra_data_ann.json", 'w')
f.write(mbra_data_ann_tolocal)
f.close()

**Construct Dataframe**

- Full dataset, containing image path, all category level names, relevant information

In [48]:
import pandas as pd

In [69]:
# Create function to process data using annotation
def create_dataset(annotation: dict, dataset: str):

    # initialize some lists for values
    img_path_array=[]
    size_array = []
    date_array = []
    latitude_array = []
    longitude_array = []

    cateid_array = []

    # Create an empty dictionary to store lists
    lists_dict = {}
    
    # 
    for i in range(len(annotation['images'])):

        # read image
        image_path = annotation['images'][i]['file_name']
        # skip this erroneoous directory (cannot read for some reason)
        if annotation['images'][i]['file_name'].startswith(("train" if dataset == 'test' else dataset) + "_mini/05212"):
            continue
        # image_pil = Image.open(image_path)
        # image_array = np.array(image_pil)
        
        # image= cv2.imread( image_path, cv2.COLOR_BGR2RGB)
        # image=cv2.resize(image, (IMG_HEIGHT, IMG_WIDTH),interpolation = cv2.INTER_AREA)
        # image=np.array(image)
        # image = image.astype('float32')
        # image /= 255 

        # fetch image information
        # fetch date
        image_date = annotation['images'][i]['date']
        # fetch location
        image_lat = annotation['images'][i]['latitude'] 
        image_long = annotation['images'][i]['longitude']
        # fetch image size
        image_size = (annotation['images'][i]['width'], annotation['images'][i]['height'])
        
        # fetch categories
        image_id = annotation['images'][i]['id']
        # check conditions
        if annotation['annotations'][i]['image_id'] != image_id:
            print('id not match')
            break
        # get category_id
        category_id = annotation['annotations'][i]['category_id']
        
        # create lists for data
        img_path_array.append(image_path)
        size_array.append(image_size)
        date_array.append(image_date)
        latitude_array.append(image_lat)
        longitude_array.append(image_long)
        # class labels
        cateid_array.append(category_id)

        ## Get all levels of gategories
        # get all category levels
        cate_types = list(annotation['categories'][0].keys())[1:-1]

        # Use a for loop to create lists and store them in the dictionary
        for type in cate_types:
            # get the current type of the current image
            cate = get_image_label(category_id, annotation['categories'], type)
            # print(type, cate)
            # print(lists_dict)
            # print(type in lists_dict)
            if type not in lists_dict:
                lists_dict[type] = [cate]
            else:
                lists_dict[type].append(cate)
        

    ## Initialize a dictionary for df
    df_dict = {
                'image_path': img_path_array
               }

    # load categories into dictionary
    for type, lst in lists_dict.items():
        df_dict[type] = lst
    
    # append remaining features
    df_dict['cate_id'] = cateid_array
    df_dict['size'] = size_array
    df_dict['date'] = date_array
    df_dict['latitude'] = latitude_array
    df_dict['longitude'] = longitude_array

    # print(len(img_path_array), len(cateid_array), len(size_array), len(date_array), len(latitude_array), len(longitude_array))
    print(len(df_dict['name']), len(df_dict['common_name']), len(df_dict['supercategory']))

    # Create a Pandas DataFrame
    df = pd.DataFrame(df_dict)

    # Write the DataFrame to a CSV file
    csv_path = f'C:\\Users\\KurtJi\\OneDrive - University of Illinois - Urbana\\Desktop\\Personal Projects\\data\\inaturalist\\data_csv\\{dataset}_dataset.csv'
    df.to_csv(csv_path, index=False)

    # return img_path_array, img_info_array, cateid_array

In [70]:
# write the dataset to local
create_dataset(mbra_data_ann, 'mbra')

110750 110750 110750


In [23]:
mbra_data_ann['categories']

[{'id': 4959,
  'name': 'Salvadora hexalepis',
  'common_name': 'western patch-nosed snake',
  'supercategory': 'Reptiles',
  'kingdom': 'Animalia',
  'phylum': 'Chordata',
  'class': 'Reptilia',
  'order': 'Squamata',
  'family': 'Colubridae',
  'genus': 'Salvadora',
  'specific_epithet': 'hexalepis',
  'image_dir_name': '04959_Animalia_Chordata_Reptilia_Squamata_Colubridae_Salvadora_hexalepis'},
 {'id': 4998,
  'name': 'Coleonyx variegatus',
  'common_name': 'Western Banded Gecko',
  'supercategory': 'Reptiles',
  'kingdom': 'Animalia',
  'phylum': 'Chordata',
  'class': 'Reptilia',
  'order': 'Squamata',
  'family': 'Eublepharidae',
  'genus': 'Coleonyx',
  'specific_epithet': 'variegatus',
  'image_dir_name': '04998_Animalia_Chordata_Reptilia_Squamata_Eublepharidae_Coleonyx_variegatus'},
 {'id': 3360,
  'name': 'Charadrius ruficapillus',
  'common_name': 'Red-capped Plover',
  'supercategory': 'Birds',
  'kingdom': 'Animalia',
  'phylum': 'Chordata',
  'class': 'Aves',
  'order': '